<a href="https://colab.research.google.com/github/vadhri/ai-notebook/blob/main/mpc/garbled_circuits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates a simple Yao's Garbled Circuit for an AND gate and a secure two-party computation protocol.


#### Simple Yao GC, AND gate.
The function below has two parts,
- A generator that creates encoded inputs and output values based on bool circuit evaluation
- An evaluator that uses the generator values and gives the ouput in encrypted or decrypted form

In [56]:
import numpy as np
import itertools
from cryptography.fernet import Fernet

def encrypt_outputs(k1, k2, value):
  k1_cipher = Fernet(k1)
  k2_cipher = Fernet(k2)
  return k2_cipher.encrypt(k1_cipher.encrypt(value))

def decrypt_outputs(k1, k2, value):
  k1_cipher = Fernet(k1)
  k2_cipher = Fernet(k2)
  return k1_cipher.decrypt(k2_cipher.decrypt(value))

def generate_io_map(no_of_values_per_input_wire, no_of_input_wires):
  io_map = {}
  labels = [[Fernet.generate_key() for _ in range(no_of_values_per_input_wire)] for _ in range(no_of_input_wires)]
  output_labels = np.array([Fernet.generate_key() for _ in range(no_of_values_per_input_wire)])
  input_combinations = itertools.product(range(no_of_values_per_input_wire), repeat=2)

  for input1, input2 in input_combinations:
    encrypted_output_label = encrypt_outputs(labels[0][input1], labels[1][input2], output_labels[input1 & input2])
    io_map[(labels[0][input1], labels[1][input2])] = encrypted_output_label

  return io_map, labels, output_labels

class GarbledAndCircuit():
  def __init__(self, io_map, labels):
    self.io_map = io_map
    self.labels = labels

  def compute(self, a, b, decrypt_output=False):
    if decrypt_output:
        return decrypt_outputs(self.labels[0][a], self.labels[1][b], self.io_map[(self.labels[0][a], self.labels[1][b])])
    return self.io_map[(self.labels[0][a], self.labels[1][b])]

# no of values is [0,1] and 2 input wires.
io_map, labels, output_labels = generate_io_map(2,2)
gc = GarbledAndCircuit(io_map, labels)

for i in range(2):
  for j in range(2):
    computed_value = gc.compute(i,j, True)
    print (f'i={i}, j={j}, output = ', np.where(output_labels == computed_value)[0][0])

i=0, j=0, output =  0
i=0, j=1, output =  0
i=1, j=0, output =  0
i=1, j=1, output =  1


#### Secure 2 party computation.

Party 0 has bit b1 and Party 1 has bit b2.

Securely compute the AND gate with 2 requirements as below.
- without either party knowing about others bit  
- both know the compute result

Step 1 : Party 0 generates input labels, output labels and an i/o map.

In [57]:
from tabulate import tabulate

io_map, input_labels, output_labels = generate_io_map(2,2)
key_labels = ["P0-key-0", "P0-key-1","P1-key-0", "P1-key-1"]

print(tabulate(zip(key_labels, [x for y in input_labels for x in y]), headers=['Key', 'Cipher']))

Key       Cipher
--------  --------------------------------------------
P0-key-0  07cFK-Rz1DI0IPofeUokmke6d3fG-qjt9Fra5xF8kOA=
P0-key-1  4CmVBCnMKQnFvLpTEWTb4CbGy-XfE15vBGvLlGeyaaw=
P1-key-0  3-C84GOesMQCYVgV2wRZdg4tGaHGqAcM0HPsXXOJ1R4=
P1-key-1  Ema-ro4AbxB-HrqT1_hVqIxRoz3_jw9AHKpef8SnjoQ=


In [58]:
for k,v in io_map.items():
  print(k,v)

(b'07cFK-Rz1DI0IPofeUokmke6d3fG-qjt9Fra5xF8kOA=', b'3-C84GOesMQCYVgV2wRZdg4tGaHGqAcM0HPsXXOJ1R4=') b'gAAAAABpBtLp313l9HqDMarPC4Q5gnt8vTj8MxeLo_yw_nGplVUtWufgqCp2u-QHs-zzpeaZX5Ed_etV-iCthnwkQUFFg3YJzwId7KbPkJtJnZ06FVAju8SP_BkTXuh6OnLO-UKuyUSuPgzS_RFfkh7C7JJWhqZPYZfIscpnVgq9pXtisHsugQVn0nrDi__wNU3S8iJmVa3jLiz-ND27dWXVqUdodKPhbBhRP5l2SPns2z48BxEiEBhWwNRwbaDyo5tGnbq-XxWP'
(b'07cFK-Rz1DI0IPofeUokmke6d3fG-qjt9Fra5xF8kOA=', b'Ema-ro4AbxB-HrqT1_hVqIxRoz3_jw9AHKpef8SnjoQ=') b'gAAAAABpBtLpWTSdtOsHjjF5-EPFFeN5C73fdB5h3-YcWWW3A4zZ6M9x-4s8Txh7z1TLefGfk3vZmRLtzm4et5P0G8Oxrroz5buP7pde2TstrEdG25hEfeNihx1o3EZBBW3SmPsjZClMYimgYfl9ssyCQOwGzyVwAAvuikIJxpPxIHluTQm9xSF2W8qRKEHnoaPZ-870eaoRX4iygHRW2zNFMGWQKjy-8oQ3rgdNjjIL07nq9gbBy-9AckTBUgXKMkUg17zXXEnC'
(b'4CmVBCnMKQnFvLpTEWTb4CbGy-XfE15vBGvLlGeyaaw=', b'3-C84GOesMQCYVgV2wRZdg4tGaHGqAcM0HPsXXOJ1R4=') b'gAAAAABpBtLpVeO1cC4MYeUH3cHa8pwxHFUiXkz12JKXp58d_sWX-vkWBb37m6djCPkPfboGhHz3gvAYLLAhDkCAiX4wF6XNVWXcMOiWYvcsTQ1nJmE2VSN0EdeAeBW9H5t5l670JInhi1E2uz_OJD4nh_Tr7

In [59]:
class BasicParallelObliviousTransfer:
  def __init__(self, party1, party2):
    self.party1 = party1
    self.party2 = party2

    self.data = None

  def send(self, data):
    self.data = data
    print(f"{self.party1} sent {data} to {self.party2}")

  def receive(self, arr_b):
    output = [self.data[arr_b]]
    return output

P0 converts it bit into cipher ad transfers the P1s K0 and K1 via oblivious transfer.

In [82]:
P0_bit = 0
P1_bit = 0

P0_input_cipher = input_labels[0][P0_bit]
print(f"Input cipher of P0 = {P0_input_cipher}")

bt = BasicParallelObliviousTransfer("P0", "P1")
bt.send(input_labels[1])

Input cipher of P0 = b'07cFK-Rz1DI0IPofeUokmke6d3fG-qjt9Fra5xF8kOA='
P0 sent [b'3-C84GOesMQCYVgV2wRZdg4tGaHGqAcM0HPsXXOJ1R4=', b'Ema-ro4AbxB-HrqT1_hVqIxRoz3_jw9AHKpef8SnjoQ='] to P1


P1 then gives their bit into OT block to find the key to be used.

In [83]:
P1_input_chiper = bt.receive(P1_bit)[0]
P1_input_chiper

b'3-C84GOesMQCYVgV2wRZdg4tGaHGqAcM0HPsXXOJ1R4='

P0 provides the following to P1.
- Key related to its bit from its lookup, P0_input_cipher (not real value)
- io map to check the output given the two keys (P0_input_cipher and P1_input_chiper)
- decrypt the value with both the keys we have
- Map decrypted value to the output result

In [84]:
output_cipher = io_map[(P0_input_cipher, P1_input_chiper)]
print (P0_input_cipher, P1_input_chiper)
encrypted_output = decrypt_outputs(P0_input_cipher, P1_input_chiper, output_cipher)
result = np.where(output_labels == encrypted_output)[0][0]
print ("Computation result = ", result)

b'07cFK-Rz1DI0IPofeUokmke6d3fG-qjt9Fra5xF8kOA=' b'3-C84GOesMQCYVgV2wRZdg4tGaHGqAcM0HPsXXOJ1R4='
Computation result =  0
